## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-06-10-53-33 +0000,nypost,"Switzerland freezes assets linked to Maduro, a...",https://nypost.com/2026/01/06/world-news/switz...
1,2026-01-06-10-49-23 +0000,bbc,Drivers over 70 to face eye tests every three ...,https://www.bbc.com/news/articles/c205nxy0p31o...
2,2026-01-06-10-40-02 +0000,nyt,Live Updates: Venezuela Isn’t Ready for Electi...,https://www.nytimes.com/live/2026/01/06/world/...
3,2026-01-06-10-36-36 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/06/world/...
4,2026-01-06-10-33-43 +0000,nypost,Tim Walz’s daughter Hope speaks out after he d...,https://nypost.com/2026/01/06/us-news/tim-walz...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,venezuela,58
4,maduro,53
22,trump,48
61,new,19
9,capture,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
194,2026-01-05-21-27-39 +0000,cbc,"Venezuela's collapse, Trump’s military buildup...",https://www.cbc.ca/news/world/trump-venezuela-...,190
228,2026-01-05-19-32-24 +0000,nyt,"Who Is Delcy Rodríguez, Venezuela’s New Interi...",https://www.nytimes.com/2026/01/05/world/ameri...,174
321,2026-01-05-12-00-00 +0000,wsj,Secretary of State Marco Rubio was there with ...,https://www.wsj.com/politics/elections/in-vene...,153
218,2026-01-05-20-11-39 +0000,nypost,Elon Musk’s Starlink offers free internet serv...,https://nypost.com/2026/01/05/business/elon-mu...,152
160,2026-01-05-22-28-00 +0000,wsj,Venezuela’s New Leader Rushes to Assert Contro...,https://www.wsj.com/world/americas/venezuela-d...,149


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
194,190,2026-01-05-21-27-39 +0000,cbc,"Venezuela's collapse, Trump’s military buildup...",https://www.cbc.ca/news/world/trump-venezuela-...
4,97,2026-01-06-10-33-43 +0000,nypost,Tim Walz’s daughter Hope speaks out after he d...,https://nypost.com/2026/01/06/us-news/tim-walz...
189,66,2026-01-05-21-36-00 +0000,wsj,U.S. companies will benefit under a new global...,https://www.wsj.com/politics/policy/oecd-corpo...
321,47,2026-01-05-12-00-00 +0000,wsj,Secretary of State Marco Rubio was there with ...,https://www.wsj.com/politics/elections/in-vene...
295,44,2026-01-05-15-07-08 +0000,nypost,"Chevron, energy stocks soar after US capture o...",https://nypost.com/2026/01/05/business/chevron...
98,42,2026-01-06-02-24-00 +0000,wsj,In the wake of the Trump administration cuttin...,https://www.wsj.com/politics/policy/trump-admi...
136,40,2026-01-05-23-50-43 +0000,nypost,Iran’s Ayatollah Ali Khamenei plans to flee to...,https://nypost.com/2026/01/05/world-news/irans...
290,38,2026-01-05-15-25-42 +0000,nypost,Paris court finds 10 people guilty of cyberbul...,https://nypost.com/2026/01/05/world-news/paris...
159,34,2026-01-05-22-35-26 +0000,nyt,Minnesota Gov. Tim Walz Abandons Re-election B...,https://www.nytimes.com/2026/01/05/us/walz-gov...
119,32,2026-01-06-00-57-00 +0000,wsj,The Pentagon is moving to demote Sen. Mark Kel...,https://www.wsj.com/politics/hegseth-moves-to-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
